# Assignment 1
## Question `1` (K-Nearest Neighbour)

| | |
|-|-|
| Course | Statistical Methods in AI |
| Release Date | `19.01.2023` |
| Due Date | `29.01.2023` |

### Instructions:
1.   Assignment must be implemented using python notebook only (Colab , VsCode , Jupyter etc.)
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for algorithms as well (sklearn etc). You are not however allowed to directly use classifier models.
3.   The performance of the model will hold weightage but you will also be graded largely for data preprocessing steps , explanations , feature selection for vectors etc.
4.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

### The Dataset
The dataset is avaible in the zip file which is a collection of *11099 tweets*. The data will be in the form of a csv file. The ground truth is also given in the zip file which corresponds to whether a tweet was popular or not. Since the task involves selecting features yourself to vectorize a tweet , we suggest some data analysis of the columns you consider important.
<br><br>

### The Task
You have to build a classifier which can predict the popularity of the tweet, i.e , if the tweet was popular or not. You are required to use **KNN** algorithm to build the classifier and cannot use any inbuilt classifier. All columns are supposed to be analyzed , filtered and preprocessed to determine its importance as a feature in the vector for every tweet (Not every column will be useful).<br>
The Data contains the **raw text of the tweet**(in the text column) as well as other **meta data** like likes count , user followers count. Note that it might be useful to **create new columns** with useful information. For example, *number of hashtags* might be useful but is not directly present as a column.<br>
There are 3 main sub parts:
1. *Vectorize tweets using only meta data* - likes , user followers count , and other created data
2. *Vectorize tweets using only it's text*. This segment will require NLP techniques to clean the text and extract a vector using a BoW model. Here is a useful link for the same - [Tf-Idf](https://towardsdatascience.com/text-vectorization-term-frequency-inverse-document-frequency-tfidf-5a3f9604da6d). Since these vectors will be very large , we recommend reducing their dimensinality (~10 - 25). Hint: [Dimentionality Reduction](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491). Please note that for this also you are allowed to use libraries.

3. *Combining the vectors from above two techinques to create one bigger vector*
<br>


Using KNN on these vectors build a classifier to predict the popularity of the tweet and report accuracies on each of the three methods as well as analysis. You can use sklearn's Nearest Neighbors and need not write KNN from scratch. (However you cannot use the classifier directly). You are expected to try the classifier for different number of neighbors and identify the optimal K value.

## Import necessary libraries

In [25]:
import pandas as pd
import numpy as np
import json
import io
import sklearn
from sklearn import preprocessing
from sklearn.metrics import accuracy_score 
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

## Load and display the data

In [26]:
df=pd.read_csv('Tweets.csv')
df_GroundTruth=pd.read_csv('ground_truth.csv', names=['Y'])

## Exploratory Data Analysis
*This is an ungraded section but is recommended to get a good grasp on the dataset*

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11099 entries, 0 to 11098
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   created_at             11099 non-null  object 
 1   id                     11099 non-null  float64
 2   id_str                 11099 non-null  float64
 3   text                   11099 non-null  object 
 4   truncated              11099 non-null  bool   
 5   entities               11099 non-null  object 
 6   metadata               11099 non-null  object 
 7   source                 11099 non-null  object 
 8   is_quote_status        11099 non-null  bool   
 9   retweet_count          11099 non-null  int64  
 10  favorite_count         11099 non-null  int64  
 11  lang                   11099 non-null  object 
 12  user_name              11099 non-null  object 
 13  user_screen_name       11099 non-null  object 
 14  user_followers_count   11099 non-null  int64  
 15  us

In [28]:
print(df.duplicated().sum())

0


In [29]:
df["truncated"].unique()

array([False,  True])

In [30]:
df["text"].unique()

array(['RT @EdwardTHardy: The 7th US Circuit Court of Appeals upheld a lower court ruling that blocked an Indiana mandate forcing women to undergoâ€¦',
       'RT @VenomMovie: The world has enough superheroes. Watch the new #Venom trailer now. 10.5.18 https://t.co/dnh8nqwkfJ',
       'RT @FutbolBible: Teachers vs Students match &amp; then one of the teachers does this... ðŸ˜±ðŸ˜±ðŸ˜± https://t.co/np6inXuAYX',
       ...,
       'Paradoxically, your good intentions may stand in the way of re... More for Capricorn https://t.co/nVhbv0cLnz',
       'RT @enblocibc: Less than 4 days to go for the International Blockchain Congress. Watch @abhi_nucleus, CEO, @NucleusVision as he tells us whâ€¦',
       'The nature of being human is to be wired for connectedness. Re... More for Pisces https://t.co/ImfXGL55qV'],
      dtype=object)

In [31]:
df.isnull().sum()

created_at               0
id                       0
id_str                   0
text                     0
truncated                0
entities                 0
metadata                 0
source                   0
is_quote_status          0
retweet_count            0
favorite_count           0
lang                     0
user_name                0
user_screen_name         0
user_followers_count     0
user_friends_count       0
user_listed_count        0
user_created_at          0
user_favourites_count    0
user_verified            0
user_statuses_count      0
dtype: int64

## Part-1
*Vectorize tweets using only meta data*

In [32]:
def get_features(df):
  """
  Funtion to return a matrix of dimensions (number of tweets, number of chosen features)
  Input parameters to this funcion are to be chosen as per requirement (Example: Loaded dataframe of the dataset) 
  """
  df2=df[['retweet_count','is_quote_status','truncated','user_verified','favorite_count','user_followers_count','user_friends_count','user_listed_count','user_favourites_count','user_statuses_count']]
  df2.loc[df2['user_verified'] == False, 'user_verified'] = 0
  df2.loc[df2['user_verified'] == True, 'user_verified'] = 1

  df2.loc[df2['truncated'] == False, 'truncated'] = 0
  df2.loc[df2['truncated'] == True, 'truncated'] = 1

  df2.loc[df2['is_quote_status'] == False, 'is_quote_status'] = 0
  df2.loc[df2['is_quote_status'] == True, 'is_quote_status'] = 1  

  return df2

  # your code here

In [33]:
df2=get_features(df)
# Normalized df2 using min-max scaler
df_norm = (df2 - df2.min()) / (df2.max() - df2.min())

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [34]:
X_train, X_test, y_train, y_test = train_test_split(df_norm,df_GroundTruth, test_size=0.20)
nbrs = NearestNeighbors(n_neighbors=5).fit(X_train,y_train)
distances, indices = nbrs.kneighbors(X_test)
indices

array([[ 849, 2589, 7957, 7488, 7665],
       [4727,  315, 5535,  494, 1308],
       [4843, 6235, 8148, 7519, 1739],
       ...,
       [7021, 6117,  105,  849, 4354],
       [6689, 3105, 8480, 3217, 1744],
       [7435, 6378, 2755, 3478, 6631]])

In [35]:
y_train=y_train.squeeze()
tmp=list(y_train)
# print(y_train.shape)

y_test=y_test.squeeze()
y_test_list=list(y_test)
result=[]

for x in indices:
  cnt0=0
  cnt1=0
  for y in x:
    if tmp[y]==0:
      cnt0=cnt0+1
    elif tmp[y]==1:
      cnt1=cnt1+1
  if cnt0>cnt1:
    result.append(0)
  else:
    result.append(1)

In [36]:
score = accuracy_score(result,y_test_list)  
print(score)  

0.9567567567567568


Perform KNN using the vector obtained from get_features() function. Following are the steps to be followed:
1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values. 

In [37]:
from urllib.parse import urlparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re

In [38]:
df=pd.read_csv('Tweets.csv')

def fun(string):
  new_string=""
  for i in string.split():
    s, n, p, pa, q, f = urlparse(i)
    if s and n:
        pass
    elif i[:1] == '@':
        pass
    elif i[:1] == '#':
        new_string = new_string.strip() + ' ' + i[1:]
    else:
        new_string = new_string.strip() + ' ' + i
  return new_string


## Part-2
Vectorize tweets based on the text. More details and reference links can be checked on the Tasks list in the start of the notebook

In [39]:
def tweet_vectoriser(df):
  """
  Funtion to return a matrix of dimensions (number of tweets, number of features extracted per tweet)
  Following are the steps for be followed:
    1. Remove links, tags and hashtags from each tweet.
    2. Apply TF-IDF on the tweets to extract a vector. 
    3. Perform dimensionality reduction on the obtained vector. 
  Input parameters to this funcion are to be chosen as per requirement (Example: Array of tweets) 
  """
  df['text'] = df['text'].apply(lambda x:fun(x))
  vectorizer = TfidfVectorizer()
  vectors=vectorizer.fit_transform(df['text'])
  df_GroundTruth=pd.read_csv('ground_truth.csv', names=['Y'])
  svd = TruncatedSVD(n_components=25)
  X_svd = svd.fit_transform(vectors)
  df4=X_svd

  return df4

In [40]:
df4 = tweet_vectoriser(df)
df_norm4 = (df4 - df4.min()) / (df4.max() - df4.min())

In [41]:
X_train, X_test, y_train, y_test = train_test_split(df_norm4,df_GroundTruth, test_size=0.20)

max_accuracy=0
optimal_k=0

for k in range(1,11,1):
  nbrs = NearestNeighbors(n_neighbors=k).fit(X_train,y_train)
  distances, indices = nbrs.kneighbors(X_test)

  y_train=y_train.squeeze()
  tmp=list(y_train)

  y_test=y_test.squeeze()
  y_test_list=list(y_test)
  result=[]

  for x in indices:
    cnt0=0
    cnt1=0
    for y in x:
      if tmp[y]==0:
        cnt0=cnt0+1
      elif tmp[y]==1:
        cnt1=cnt1+1
    if cnt0>cnt1:
      result.append(0)
    else:
      result.append(1)

  score = accuracy_score(result,y_test_list)
  if max_accuracy<=score:
    max_accuracy=score
    optimal_k=k


print(max_accuracy)
print(optimal_k) 

0.9315315315315316
9


Perform KNN using the vector obtained from tweet_vectoriser() function. Following are the steps to be followed:

1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values.

In [42]:
max_accuracy=0
optimal_k=0

for k in range(1,11,1):
  nbrs = NearestNeighbors(n_neighbors=k).fit(X_train,y_train)
  distances, indices = nbrs.kneighbors(X_test)


  y_train=y_train.squeeze()
  tmp=list(y_train)
  # print(y_train.shape)

  y_test=y_test.squeeze()
  y_test_list=list(y_test)
  result=[]

  for x in indices:
    cnt0=0
    cnt1=0
    for y in x:
      if tmp[y]==0:
        cnt0=cnt0+1
      elif tmp[y]==1:
        cnt1=cnt1+1
    if cnt0>cnt1:
      result.append(0)
    else:
      result.append(1)

  score = accuracy_score(result,y_test_list)
  if max_accuracy<=score:
    max_accuracy=score
    optimal_k=k

print(max_accuracy)
print(optimal_k) 

0.9315315315315316
9


## Part-3
### Subpart-1

Combine both the vectors obtained from the tweet_vectoriser() and get_features()

In [43]:
# your code here
df5=pd.DataFrame(df4)
df6=pd.concat([df2,df5],axis=1)

df_norm6 = (df6 - df6.min()) / (df6.max() - df6.min())
X_train, X_test, y_train, y_test = train_test_split(df_norm6,df_GroundTruth, test_size=0.20)

Perform KNN using the vector obtained in the previous step. Following are the steps to be followed:

1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values.

In [44]:
max_accuracy=0
optimal_k=0

for k in range(1,11,1):
  nbrs = NearestNeighbors(n_neighbors=k).fit(X_train,y_train)
  distances, indices = nbrs.kneighbors(X_test)

  y_train=y_train.squeeze()
  tmp=list(y_train)

  y_test=y_test.squeeze()
  y_test_list=list(y_test)
  result=[]

  for x in indices:
    cnt0=0
    cnt1=0
    for y in x:
      if tmp[y]==0:
        cnt0=cnt0+1
      elif tmp[y]==1:
        cnt1=cnt1+1
    if cnt0>cnt1:
      result.append(0)
    else:
      result.append(1)

  score = accuracy_score(result,y_test_list)
  if max_accuracy<=score:
    max_accuracy=score
    optimal_k=k

print(max_accuracy)
print(optimal_k) 

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

0.9346846846846847
7


### Subpart-2

Explain the differences between the accuracies obtained in each part above based on the features used.

1.1 :

Firstly,I have used all float features like retweet_count, is_quote_status, truncated, user_verified ,favorite_count, user_followers_count, user_friends_count, user_listed_count, user_favourites_count, user_statuses_count by which I got accuracy of 95.54054054054054 %.

After doing some more experiments on which features to choose ,I got following features which gives highest of 96.03603603603603 % of accuracy.
Features are retweet_count, is_quote_status, truncated, user_verified ,favorite_count, user_followers_count, user_friends_count, user_listed_count, user_favourites_count, user_statuses_count,user_verified,truncated,is_quote_status

1.2 :

Used Feature "text" from Dataset and applied NearestNeighbors for a range of k values ranging from 1 to 10.
Best accuracy achieved is around 93 to 95 % 

1.3 :

Used Feature "text" from Dataset and applied NearestNeighbors for a range of k values ranging from 1 to 10.
Best accuracy achieved is around 93 to 95 % 